## 리스코프 치환 원칙(LSP)  

- 설계 시 안정성을 유지하기 위해 객체 타입이 유지해야하는 일련의 특성   
    --> 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있어야 한다는 것으로 어떤 하위타입을 사용해도 실행에 따른 결과를 염려하지 않아야 한다는 것.   \

LISKOV 01 : S가 T의 하위타입이라면 프로그램을 변경하지 않고 T 타입의 객체를 S 타입의 객체로 치환 가능해야 한다.  

### 도구를 사용해 LSP 문제 검사하기.  

Mypy나 Pylint와 같은 도구를 사용해 쉽게 검출할 수 있다.  

#### 메서드 서명의 잘못된 데이터타입 검사  

코드 전체에 타입 어노테이션을 사용했고, Mypy를 설정했다면 초기에 기본 오류 여부와 LSP 준수 여부를 빠르게 파악할 수 있다.

In [4]:
class Event:
    def meets_condition(self, event_data: dict) -> bool:
        return False


class LoginEvent(Event):
    def meets_condition(self, event_data: list) -> bool:
        return bool(event_data)

위의 코드는 LSP 위반의 기준이 명확하며, 파생 클래스가 부모 클래스에서 정의한 파라메타와 다른 타입을 사용했기 때문에 다르게 동작한다.  
   --> 호출자는 아무런 차이를 느끼지 않고 투명하게 Event 또는 LoginEvent를 사용할 수 있어야 하며, 2가지 타입의 객체를 치환해도 앱 실행에 실패해서는 안된다.  

반환 값을 Bool 값이 아닌 다른 값으로 변경해도 동일한 오류가 발생하며, 오류의 이론적 근거는 클라이언트가 Bool 값을 사용할 것으로 기대한다는 것이다.  


#### Pylint로 호환되지 않는 서명 검사  

메서드의 서명 자체가 완전히 다른 경우에도 발생. 
밑의 코드처럼 기존 Event 내 함수의 파라메타가 LogoutEvent 클래스에서 파라메타가 추가되게 되도, 오류를 감지하는 것이 쉽지 않다.  

In [5]:
class LogoutEvent(Event):
    def meets_condition(self, event_data: dict, override: bool) -> bool:
        if override:
            return True

### 애매한 LSP 위반 사례  

- 하위 클래스는 부모 클래스에 정의된 것보다 사전조건을 엄격하게 만들면 안된다.  
- 하위 클래스는 부모 클래스에 정의된 것보다 약한 사후조건을 만들면 안된다.  

In [6]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False

    @staticmethod
    def meets_condition_pre(event_data: dict):
        assert isinstance(event_data, dict), f"{event_data!r} is not a dict"
        for moment in ("before", "after"):
            assert moment in event_data, f"{moment} not in {event_data}"
            assert isinstance(event_data[moment], dict)

In [11]:
class SystemMonitor:
    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        Event.meets_condition(self.event_data)
        event_cls = next((event_cls for event_cls in Event.__subclasses__() if event_cls.meets_condition(self.event_data)), UnknownEvent, )
        return event_cls(self.event_data)


class TransactionEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None


class UnknownEvent(Event):
    pass
        

In [12]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__

'UnknownEvent'

LSP는 다형성을 강조하기 때문에 좋은 디자인의 기초가 되며, 올바른 계층구조를 갖도록 하여 상속된 클래스가 부모 클래스와 다형성을 유지하도록 한다.  


## 인터페이스 분리 원칙   

ISP(Interface Segregation Principle)은 작은 인터페이스에 대한 가이드 라인을 제공한다.  

인터페이스 : 객체가 노출하는 메서드의 집합.  
    --> 인터페이스는 클래스 메서드의 형태를 보고 암시적으로 정의하는데 파이썬의 덕 타이핑(Duck Typing) 원리를 따르기 때문이다.  
- 덕 타이핑 : 모든 객체가 자신이 가지고 있는 메서드와 자신이 할 수 있는 일에 의해서 표현된다는 점에서 출발한다.  
- 다중 메서드를 가진 인터페이스가 있다면 매우 정확하고 구체적인 구분에 따라 더 적은 수의 메서드를 가진 여러 개의 메서드로 분할하는 것이 좋다.  
- 재사용성을 높이기 위해 가능한 작은 단위로 인터페이스를 분리한다면 인터페이스 중 하나를 구현하려는 각 클래스가 매우 명확한 동작과 책임을 지니기 때문에 응집력이 높아진다.  


### 너무 많은 일을 하는 인터페이스  

추상 기본 클래스 내 여러 메서드를 정의하게 되면 파생클래스에서 사용하지 않는 메서드도 정의를 해야하는 문제가 발생하며 결합력을 높이고 유연성을 떨어트린다.  


### 인터페이스는 작을수록 좋다. 

여러 메서드를 목적에 맞게 여러 인터페이스로 분리하게 되면, 독립성을 유지하게 되고 새로운 작은 객체를 사용해 모든 기능을 유연하게 조합할 수 있게 된다.  


### 인터페이스는 얼마나 작아야 할까?  

추상클래스든 아니든 기본 클래스는 다른 클래스들이 확장할 수 있도록 인터페이스를 정의한다.  
    --> 하나 이상의 메서드라 하더라도 적절하게 하나의 클래스에 속해 있을 수 있는데, 하나의 클래스에서 제공하지 않으면 잘못 이해할 가능성이 높은 것들은 여러 메서드를 추가해야 한다.  


## 의존성 역전  

의존성 역전 원칙(DIP) : 코드가 깨지거나 손상되는 취약점으로부터 보호해주는 흥미로운 디자인 원칙 제시.  
    --> 코드가 세부 사항이나 구체적인 구현에 적응하도록 하지 않고, API 같은 것에 적응하도록 하는 것!  

추상화는 인터페이스 형태로 제공되는데, 일반적으로 구체적인 구현이 추상 컴포넌트보다 훨씬 더 자주 바뀔 것이기 때문에, 시스템이 변경, 수정 또는 확장될 것으로 예상되는 지점에 유연성을 확보하기 위해 추상화를 하는 것이다.  


### 엄격한 의존의 예  
### 의존성을 거꾸로

- 추상 기본 클래스를 사용하는 것은 좋은 습관이다.
1. 덕 타이핑 : 모델 가독성이 올라간다.   
2. 상속은 is a 관계임을 기억해야 한다.


## 요약

SOLID 원칙은 객체 지향 소프트웨어 설계의 핵심 원칙이며, 소프트웨어 빌드는 엄청나게 어려운 작업이고, 코드의 논리는 복잡하다.  

- 디자인을 잘못하게 되면 미래에 많은 비용이 든다는 것을 의미하므로, 원칙에 충실하게 소프트웨어를 만들 수 있도록 도와야 한다.  
